In [1]:
#%load_ext tensorboard

In [1]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
# import jax.profiler
# server = jax.profiler.start_server(9999)
%load_ext line_profiler

In [2]:
from emlp_jax.mlp import MLP,EMLP#,LinearBNSwish
from emlp_jax.datasets import Fr
import jax.numpy as jnp
import jax
import optax

In [3]:
from emlp_jax.equivariant_subspaces import T,Scalar,Matrix,Vector,Quad,repsize
from emlp_jax.groups import SO,O,Trivial,Lorentz,O13,SO13,SO13p
from emlp_jax.mlp import EMLP,LieLinear
import itertools
import numpy as np
import torch
from emlp_jax.datasets import Inertia,Fr,ParticleInteraction
import objax

In [4]:
import torch
from torch.utils.data import DataLoader
from slax.utils.utils import LoaderTo,cosLr, islice, export,FixedNumpySeed
from slax.tuning.study import train_trial
from slax.datasetup.datasets import split_dataset
from slax.model_trainers.classifier import Regressor
from functools import partial
import torch.nn as nn
#repmiddle = 100*T(0)+30*T(1)+10*T(2)+3*T(3)#+1*T(4)

def makeTrainer(*,dataset=Fr,network=EMLP,num_epochs=500,ndata=1000+1000,seed=2020,aug=False,
                bs=500,lr=1e-2,device='cuda',split={'train':-1,'test':1000},
                net_config={'num_layers':3,'group':SO(3)},
                trainer_config={'log_dir':None,'log_args':{'minPeriod':.02}},save=False):

    # Prep the datasets splits, model, and dataloaders
    with FixedNumpySeed(seed):
        datasets = split_dataset(dataset(ndata),splits=split)
    device = torch.device(device)
    model = network(datasets['train'].rep_in,datasets['train'].rep_out,**net_config)
    dataloaders = {k:LoaderTo(DataLoader(v,batch_size=bs,shuffle=(k=='train'),
                num_workers=0,pin_memory=False)) for k,v in datasets.items()}
    dataloaders['Train'] = dataloaders['train']
    opt_constr = objax.optimizer.Adam
    lr_sched = lambda e: lr*cosLr(num_epochs)(e)
    return Regressor(model,dataloaders,opt_constr,lr_sched,**trainer_config)

In [5]:
trainer = makeTrainer(network=EMLP,lr=3e-3)
#trainer = makeTrainer(network=MLP,lr=1e-3,aug=False)


INFO:root:Linear W components:2592 shape:(432, 6) rep:300T(1)+68T(2)+22T(3)+6T(4) @ d=3
INFO:root:BiW components:2592 dim:600 shape:(432, 6) rep:300T(1)+68T(2)+22T(3)+6T(4) @ d=3
INFO:root:Linear W components:165888 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
INFO:root:BiW components:165888 dim:65568 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
INFO:root:Linear W components:165888 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
INFO:root:BiW components:165888 dim:65568 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
INFO:root:Linear W components:384 shape:(1, 384) rep:102T(0)+34T(1)+11T(2)+3T(3) @ d=3


In [6]:
#trainer.train(500)

train:   0%|          | 0/500 [00:00<?, ?it/s]

     test_MSE  Train_MSE
-2  24.757954   22.91957
     test_MSE  Train_MSE
101  0.109429   0.094671
     test_MSE  Train_MSE
161  0.072954   0.066781
     test_MSE  Train_MSE
209  0.010102   0.008751
     test_MSE  Train_MSE
248   0.01648   0.010867
     test_MSE  Train_MSE
282  0.024221   0.023751
     test_MSE  Train_MSE
312  0.024074   0.018698
     test_MSE  Train_MSE
341  0.006749   0.005236
     test_MSE  Train_MSE
368  0.029189   0.026884
     test_MSE  Train_MSE
395  0.008284   0.005669
     test_MSE  Train_MSE
420  0.027824   0.030762
     test_MSE  Train_MSE
444  0.008767   0.005437
     test_MSE  Train_MSE
467  0.004948    0.00442
     test_MSE  Train_MSE
490  0.003309   0.001762
     test_MSE  Train_MSE
513   0.00777   0.006903
     test_MSE  Train_MSE
536  0.004741   0.003132
     test_MSE  Train_MSE
558  0.016887   0.016715
     test_MSE  Train_MSE
580  0.016959   0.017056
     test_MSE  Train_MSE
601  0.021002   0.018966
     test_MSE  Train_MSE
622  0.007727   0.005047


In [5]:
from emlp_jax.mlp import uniform_rep

In [9]:
uniform_rep(256,Lorentz)

64T(0, 0)+8T(0, 1)+8T(1, 0)+T(0, 2)+2T(1, 1)+T(2, 0)+T(3, 0) @ d=4

Initialization considerable better in pytorch? Why else does the loss go down so much faster in pytorch. Could be the adam betas? Or perhaps clamp vs rsqrt.

In [19]:
logging.getLogger().setLevel(logging.INFO)

In [20]:
# The apply function now takes both params **and** state. Additionally it will
# return updated values for state. In the resnet example this will be the
# updated values for moving averages used in the batch norm layers.
logits, state = net.apply(params, state, x)

bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)


In [16]:
state

FlatMapping({
  'emlp/~/tensor_mask_bn/~/mean_ema': FlatMapping({
                                        'counter': DeviceArray(1, dtype=int32),
                                        'hidden': DeviceArray([-2.72169262e-02,  3.35316695e-02, -2.94104172e-03,
                                                                6.48176298e-03, -3.12723741e-02, -2.12527718e-02,
                                                               -1.36882742e-03, -1.15396166e-02, -2.69303899e-02,
                                                                2.59548817e-02, -8.30877572e-03, -4.91703255e-03,
                                                               -3.85290794e-02,  1.22720916e-02,  1.94078311e-02,
                                                               -2.23876797e-02,  1.66982170e-02, -2.91648041e-03,
                                                               -2.40416378e-02,  2.69001815e-02, -1.66822281e-02,
                                                        

In [11]:
%debug

> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(4326)_canonicalize_tuple_index()
   4324   if len_without_none > arr_ndim:
   4325     msg = "Too many indices for array: {} non-None/Ellipsis indices for dim {}."
-> 4326     raise IndexError(msg.format(len_without_none, arr_ndim))
   4327   ellipses = (i for i, elt in enumerate(idx) if elt is Ellipsis)
   4328   ellipsis_index = next(ellipses, None)

ipdb> up
> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(4047)_index_to_gather()
   4045 def _index_to_gather(x_shape, idx):
   4046   # Remove ellipses and add trailing slice(None)s.
-> 4047   idx = _canonicalize_tuple_index(len(x_shape), idx)
   4048 
   4049   # Check for advanced indexing:

ipdb> up
> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(3963)_gather()
   3961 def _gather(arr, treedef, static_idx, dynamic_idx):
   3962   idx = _merge_static_and_dyn

In [1]:
import jax
#jax.config.update('jax_platform_name', 'cpu')
from emlp_jax.equivariant_subspaces import T
from emlp_jax.utils import NoCache as nocache
from emlp_jax.groups import SO,O,C,D,Scaling,Parity,TimeReversal,Lorentz,SO13p,SO13,O13,Symplectic,Permutation,Trivial
# with nocache():
#     for i in range(6):
#         for j in range(i+1):
#             print(f"T{i-j,j}: {T(i-j,j)(SO13).symmetric_subspace()[0]}")

In [6]:
#with nocache():
T(7)(SO13p()).symmetric_subspace()[0] 

RuntimeError: Resource exhausted: Out of memory while trying to allocate 6442450944 bytes.

In [1]:
from emlp_jax import mixed_tensors
from emlp_jax.equivariant_subspaces import *
from emlp_jax.groups import *


In [2]:
(T(1)+2*T(2)+T(0))(SO(3))*T(1)(S(5))

T(1)⊗T(1)+2T(2)⊗T(1)+T(0)⊗T(1)

In [3]:
((T(1)+5*T(2))*(T(1)+5*T(2)))(S(3))

T(2)+25T(3)+25T(4)

In [2]:
import jax
#jax.config.update('jax_platform_name', 'cpu')
import numpy as np
import jax.numpy as jnp
from emlp_jax.equivariant_subspaces import T
from emlp_jax.groups import *


In [20]:
(T(1)+T(2))(S(40)).symmetric_basis().shape

Krylov Solving for Equivariant Subspace r<=10:   0%|          | 0/1000 [00:00<?, ?it/s]

(2, 1600)

In [30]:
T(3)(RubiksCube()).symmetric_basis().shape

Krylov Solving for Equivariant Subspace r<=10:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace r<=20:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace r<=40:   0%|          | 0/1000 [00:00<?, ?it/s]

(22, 110592)

In [29]:
48**4

5308416

In [31]:
T(3)(S(48)).symmetric_basis().shape

Krylov Solving for Equivariant Subspace r<=10:   0%|          | 0/1000 [00:00<?, ?it/s]

(5, 110592)

In [32]:
from emlp_jax.datasets import BrokenRubiksCube
from emlp_jax.mlp import uniform_allocation

In [43]:
T(1)(RubiksCube()).symmetric_basis()

DeviceArray([[ 0.20369887, -0.01316992,  0.20369883, -0.01316992,
              -0.01316992,  0.2036988 , -0.01316993,  0.20369881,
               0.20369884, -0.01316993,  0.20369887, -0.01316994,
              -0.01316993,  0.20369883, -0.01316993,  0.20369895,
               0.20369884, -0.01316994,  0.2036989 , -0.01316993,
              -0.01316994,  0.20369883, -0.01316993,  0.2036988 ,
               0.20369884, -0.01316995,  0.20369884, -0.01316993,
              -0.01316994,  0.20369884, -0.01316993,  0.20369881,
               0.20369884, -0.01316993,  0.2036989 , -0.01316994,
              -0.01316994,  0.20369887, -0.01316994,  0.20369886,
               0.20369886, -0.01316994,  0.20369886, -0.01316993,
              -0.01316994,  0.20369878, -0.01316994,  0.20369883],
             [ 0.01316993,  0.20369881,  0.01316993,  0.20369875,
               0.20369874,  0.01316992,  0.20369866,  0.01316993,
               0.01316994,  0.20369878,  0.01316996,  0.20369881,
         

In [40]:
rep = (100*T(0)+5*T(1)+T(2))(RubiksCube())

In [41]:
rep.size()

2644

In [10]:
ds = BrokenRubiksCube(False)

In [14]:
ds.X.shape

(240, 6, 48)

In [12]:
(ds.X.reshape((-1,6,48))*np.arange(6)[None,:,None]).sum(1)

DeviceArray([[0., 0., 5., ..., 5., 5., 5.],
             [0., 0., 0., ..., 2., 5., 5.],
             [4., 4., 4., ..., 5., 5., 2.],
             ...,
             [2., 0., 0., ..., 2., 5., 0.],
             [5., 1., 1., ..., 5., 5., 4.],
             [2., 5., 5., ..., 1., 0., 0.]], dtype=float32)

In [2]:
G =RubiksCube()

In [3]:
Q = T(2)(G).symmetric_basis()

Krylov Solving for Equivariant Subspace r<=10:   0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
Ms = Q.reshape(Q.shape[0],48,48)

In [5]:
g = G.samples(Ms.shape[0])

In [11]:
print(Ms.shape,g.shape)

(6, 48, 48) (6, 48, 48)


In [8]:
import jax.numpy as jnp
def rel_err(a,b):
    return jnp.sqrt(((a-b)**2).mean())/(jnp.sqrt((a**2).mean())+jnp.sqrt((b**2).mean()))

In [12]:
rel_err(g@Ms,Ms@g)

DeviceArray(0.00052345, dtype=float32)

In [14]:
v = T(1)(G).symmetric_basis()
g2 = G.samples(v.shape[0])
rel_err(g2@v[:,:,None],v[:,:,None])

DeviceArray(1.9200611e-07, dtype=float32)

In [6]:
G = S(10)
T(5)(G).symmetric_basis().shape[0]

Krylov Solving for Equivariant Subspace r<=10:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace r<=20:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace r<=40:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace r<=80:   0%|          | 0/1000 [00:00<?, ?it/s]

52

In [3]:
G.discrete_generators

DeviceArray([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

             [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1.,

In [4]:
G = RubiksCube()
T(4)(G).symmetric_basis().shape[0]

Krylov Solving for Equivariant Subspace:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace:   0%|          | 0/1000 [00:00<?, ?it/s]

Krylov Solving for Equivariant Subspace:   0%|          | 0/1000 [00:00<?, ?it/s]

RuntimeError: Resource exhausted: Out of memory while trying to allocate 10192158768 bytes.

In [4]:
#T(1)(G).show_subspace()

In [3]:
T(2)(G).symmetric_basis().shape[0]

6

In [2]:
T(2)(S(48)).symmetric_basis().shape[0]

RuntimeError: Resource exhausted: Failed to allocate request for 43.68GiB (46905163776B) on device ordinal 0

In [8]:
U,F,R,B,L,D = G.discrete_generators

In [9]:
(U@U@U@U==np.eye(48)).all()

True

In [10]:
M = R@U@U@D@D@D@B@D@D@D

In [22]:
M = D@D@D@B@D@D@D@U@U@R

(RU^{2}D^{-1}BD^{-1})

In [23]:
M1260 = jnp.linalg.matrix_power(M,1260)
(M1260==np.eye(48)).all()

DeviceArray(True, dtype=bool)

In [21]:
jnp.abs(M1260-np.eye(48)).sum()

DeviceArray(16., dtype=float32)

In [4]:
p1 = np.array([1,0,3,2])
p2 = np.array([0,1,3,2])
p3 = np.array([3,0,2,1])

In [5]:
x = np.arange(4)

In [6]:
x[p1][p2][p3]

array([3, 1, 2, 0])

In [7]:
x[p1[p2[p3]]]

array([3, 1, 2, 0])

In [12]:
np.eye(4)[p1]@x

array([1., 0., 3., 2.])

In [13]:
x[p1]

array([1, 0, 3, 2])